In [1]:
import numpy as np
import pandas as pd
import os
file1="/Users/zhilinyang/Desktop/Disk_SMART_dataset.txt"
file2="/Users/zhilinyang/Desktop/dataset.txt"

Each line in the dataset contains 14 columns which are seperated by commas. The meaning of each column is listed as follows.
Column 1 : index of the disk representing it's serial-number, ranging from 1 to 23,395.
Column 2 : class label of the disk, it is -1 for disks that are failed and +1 for disks that are good.
Column 3 : VALUE of SMART ID #1 , Raw Read Error Rate
Column 4 : VALUE of SMART ID #3 , Spin Up Time
Column 5 : VALUE of SMART ID #5 , Reallocated Sectors Count
Column 6 : VALUE of SMART ID #7 , Seek Error Rate
Column 7 : VALUE of SMART ID #9 , Power On Hours
Column 8 : VALUE of SMART ID #187 , Reported Uncorrectable Errors
Column 9 : VALUE of SMART ID #189 , High Fly Writes
Column 10 : VALUE of SMART ID #194 , Temperature Celsius
Column 11 : VALUE of SMART ID #195 , Hardware ECC Recovered
Column 12 : VALUE of SMART ID #197 , Current Pending Sector Count
Column 13 : RAW_VALUE of SMART ID #5 , Reallocated Sectors Count
Column 14 : RAW_VALUE of SMART ID #197 , Current Pending Sector Count

In [10]:
with open(file1,"r") as f1, open(file2,"w") as f2:
    content=f1.read()
    lines=content.split("\n")
    for line in lines:
        datas=line.split(", ")
        newline=" ".join(datas)
        f2.write(newline+'\n')
print('done')   

done


In [9]:
colname=["SID","label","SMART_1","SMART_3","SMART_5","SMART_7","SMART_9","SMART_187",
         "SMART_189","SMART_194","SMART_195","SMART_197","RAW_5","RAW_197"]

In [10]:
df=pd.read_table(file2,sep=' ')
df.columns=colname
df.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,1,-1,0.483871,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0
1,1,-1,0.000000,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0
2,1,-1,0.096774,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.714286,-0.139785,1.0,0.935255,-1.0
3,1,-1,0.193548,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.139785,1.0,0.935255,-1.0
4,1,-1,0.290323,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0


In [11]:
good_disks=df[df.label==1]
fail_disks=df[df.label==-1]
print(len(good_disks),len(fail_disks))

3850141 156311


In [12]:
good_disks_num=len(good_disks["SID"].unique())
fail_disks_num=len(fail_disks["SID"].unique())
print("num of good disks: %d, bad disks: %d" % (good_disks_num, fail_disks_num))

num of good disks: 22962, bad disks: 433


In [39]:
def get_lastn_samples(total_df,new_df,n,disk_num):
    groups=total_df.groupby('SID')
    disk_sid=list(total_df['SID'].unique())
    for i in range(0,disk_num):
        obj=groups.get_group(disk_sid[i])
        if len(obj)<n:
            continue
        obj=obj.iloc[len(obj)-n:len(obj)]
        new_df=new_df.append(obj)
    return new_df

In [31]:
fail_samples=pd.DataFrame()
fail_samples=get_lastn_samples(fail_disks,fail_samples,12,fail_disks_num)
fail_samples.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
128,1,-1,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
129,1,-1,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
130,1,-1,0.483871,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
131,1,-1,-0.064516,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.247312,1.0,0.935255,-1.0
132,1,-1,0.032258,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.600000,-0.247312,1.0,0.935255,-1.0


In [93]:
fail_list=pd.DataFrame(fail_samples['SID'].unique())
train_list=fail_list.sample(frac=0.7).values

In [97]:
groups=fail_samples.groupby('SID')
for sid in fail_samples['SID'].unique():
    if sid in train_list:
        obj=groups.get_group(sid)
        fail_train=fail_train.append(obj)
fail_train.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
1097,4,-1,-0.096774,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
1098,4,-1,0.161290,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
1099,4,-1,0.290323,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
1100,4,-1,0.354839,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
1101,4,-1,0.387097,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0


In [34]:
fail_test=pd.DataFrame()
groups=fail_samples.groupby('SID')
train_list=list(fail_train['SID'].unique())
for sid in fail_samples['SID'].unique():
    if sid not in train_list:
        obj=groups.get_group(sid)
        fail_test=fail_test.append(obj)
fail_test.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
128,1,-1,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
129,1,-1,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
130,1,-1,0.483871,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.182796,1.0,0.935255,-1.0
131,1,-1,-0.064516,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,-0.247312,1.0,0.935255,-1.0
132,1,-1,0.032258,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.600000,-0.247312,1.0,0.935255,-1.0


In [ ]:
fail_train.to_csv('/Users/zhilinyang/Desktop/fail_train.csv')

In [37]:
print(len(fail_train),len(fail_test))

3636 1560


In [2]:
fail_train=pd.read_csv('/Users/zhilinyang/Desktop/fail_train.csv')
fail_train.head()

,Unnamed: 0,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,1097,4,-1,-0.096774,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
1,1098,4,-1,0.161290,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
2,1099,4,-1,0.290323,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
3,1100,4,-1,0.354839,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
4,1101,4,-1,0.387097,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0


In [7]:
fail_train=fail_train.iloc[:,1:]
fail_train.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,4,-1,-0.096774,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
1,4,-1,0.161290,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.600000,-0.096774,1.0,-1.0,-1.0
2,4,-1,0.290323,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
3,4,-1,0.354839,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0
4,4,-1,0.387097,0.777778,1.0,0.439024,0.368421,1.0,0.959596,-0.542857,-0.075269,1.0,-1.0,-1.0


In [8]:
fail_train=fail_train.dropna()
len(fail_train)

3636

In [32]:
def get_healthy_train(total_df,train_df):
    name_list=total_df['SID'].unique()
    x=np.array(name_list)
    it = np.nditer(x,flags=['multi_index','refs_ok'])
    groups=total_df.groupby('SID')
    while not it.finished:
        ix=it.multi_index
        obj=groups.get_group(x[ix])
        train_size=int(len(obj)*0.7)
        obj=obj.iloc[0:train_size+1]
        samples=obj.sample(3)
        train_df=train_df.append(samples)
        it.iternext()
    return train_df

In [36]:
def get_healthy_test(total_df,train_df):
    name_list=total_df['SID'].unique()
    x=np.array(name_list)
    it = np.nditer(x,flags=['multi_index','refs_ok'])
    groups=total_df.groupby('SID')
    while not it.finished:
        ix=it.multi_index
        obj=groups.get_group(x[ix])
        train_size=int(len(obj)*0.7)
        obj=obj.iloc[train_size+1:]
        if (len(obj)<3):
            it.iternext()
            continue
        samples=obj.sample(3)
        train_df=train_df.append(samples)
        it.iternext()
    return train_df

In [28]:
healthy_train=pd.DataFrame()
healthy_train=get_healthy_train(good_disks,healthy_train)
healthy_train.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
156326,434,1,0.387097,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.485714,-0.118280,1.0,-0.964083,-1.0
156312,434,1,0.322581,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.485714,-0.161290,1.0,-0.964083,-1.0
156320,434,1,0.451613,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,-0.182796,1.0,-0.964083,-1.0
156539,435,1,0.258065,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.485714,-0.075269,1.0,-0.996692,-1.0
156553,435,1,0.096774,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.428571,-0.247312,1.0,-0.996692,-1.0


In [29]:
len(healthy_train)

68886

In [25]:
healthy_test=pd.DataFrame()
healthy_test=get_healthy_test(good_disks,healthy_test)
healthy_test.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
156474,434,1,0.387097,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.600000,-0.182796,1.0,-0.964083,-1.0
156449,434,1,0.193548,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.600000,-0.053763,1.0,-0.964083,-1.0
156465,434,1,-0.032258,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.600000,-0.161290,1.0,-0.964083,-1.0
156643,435,1,0.451613,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.485714,-0.075269,1.0,-0.996692,-1.0
156622,435,1,0.322581,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.542857,-0.225806,1.0,-0.996692,-1.0


In [26]:
len(healthy_test)

68886

In [38]:
train_df=healthy_train.append(fail_train)
train_df=train_df.sample(frac=1)
y_train=train_df['label']
train_df=train_df.drop('label',axis=1)
train_df.head()

,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
804092,4294,0.483871,1.0,1.000000,0.512195,0.157895,1.0,1.000000,-0.371429,-0.290323,1.0,-0.995274,-1.0
749349,3967,0.129032,1.0,0.818182,0.560976,0.052632,1.0,1.000000,-0.542857,-0.182796,1.0,-0.811437,-1.0
1033062,5658,0.451613,1.0,1.000000,0.560976,0.157895,1.0,1.000000,-0.200000,-0.182796,1.0,-0.998582,-1.0
2578996,14870,0.290323,1.0,1.000000,0.414634,0.578947,1.0,1.000000,-0.428571,-0.139785,1.0,-1.000000,-1.0
162630,471,-0.129032,1.0,1.000000,0.536585,0.052632,1.0,-0.434343,-0.714286,-0.053763,1.0,-0.997637,-1.0


In [ ]:
y_train[y_train==1]=0
y_train[y_train==-1]=1
y_train

In [42]:
test_df=healthy_test.append(fail_test)
test_df=test_df.sample(frac=1)
y_test=test_df['label']
test_df=test_df.drop('label',axis=1)
test_df.head()

,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
2090696,11962,0.419355,0.555556,1.0,0.390244,0.578947,1.0,1.000000,-0.542857,-0.053763,1.0,-1.00000,-1.0
3614931,21057,0.451613,-0.185185,1.0,0.292683,0.789474,1.0,1.000000,-0.371429,-0.204301,1.0,-0.99811,-1.0
1703073,9654,0.290323,1.000000,1.0,0.243902,0.578947,1.0,1.000000,-0.200000,-0.118280,1.0,-1.00000,-1.0
3359406,19535,0.193548,0.111111,1.0,0.317073,0.789474,1.0,1.000000,-0.485714,-0.161290,1.0,-1.00000,-1.0
2063986,11803,0.290323,1.000000,1.0,0.390244,0.578947,1.0,0.959596,-0.600000,-0.010753,1.0,-1.00000,-1.0


In [ ]:
y_test[y_test==1]=0
y_test[y_test==-1]=1
y_test

In [12]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='entropy',min_samples_split = 20, min_samples_leaf = 7, min_impurity_decrease = 0.001)

In [13]:
DT.fit(train_df.iloc[:,1:],y_train.values)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
DT.score(test_df.iloc[:,1:],y_test.values)

0.99388183857138801

In [15]:
y_pred=DT.predict(test_df.iloc[:,1:])

In [16]:
from sklearn.metrics import f1_score,precision_score
score=f1_score(y_test,y_pred)
pres=precision_score(y_test,y_pred)
print(score,pres)

0.867993874426 0.831085043988


In [17]:
from sklearn.metrics import confusion_matrix
def FDR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    TP=mat[1,1]
    FN=mat[1,0]
    FDR=TP/(TP+FN)
    return FDR

In [18]:
FDR(y_test,y_pred)

0.90833333333333333

In [53]:
good_disks.to_csv('/Users/zhilinyang/Desktop/work/good_disks.csv')
fail_disks.to_csv('/Users/zhilinyang/Desktop/work/fail_disks.csv')

In [2]:
train_df=pd.read_csv('/Users/zhilinyang/Desktop/work/train_df.csv')
train_df.head()

,Unnamed: 0,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,804092,4294,0.483871,1.0,1.000000,0.512195,0.157895,1.0,1.000000,-0.371429,-0.290323,1.0,-0.995274,-1.0
1,749349,3967,0.129032,1.0,0.818182,0.560976,0.052632,1.0,1.000000,-0.542857,-0.182796,1.0,-0.811437,-1.0
2,1033062,5658,0.451613,1.0,1.000000,0.560976,0.157895,1.0,1.000000,-0.200000,-0.182796,1.0,-0.998582,-1.0
3,2578996,14870,0.290323,1.0,1.000000,0.414634,0.578947,1.0,1.000000,-0.428571,-0.139785,1.0,-1.000000,-1.0
4,162630,471,-0.129032,1.0,1.000000,0.536585,0.052632,1.0,-0.434343,-0.714286,-0.053763,1.0,-0.997637,-1.0


In [3]:
train_df=train_df.iloc[:,1:]
test_df=pd.read_csv('/Users/zhilinyang/Desktop/work/test_df.csv')
test_df=test_df.iloc[:,1:]
test_df.head()

,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,11962,0.419355,0.555556,1.0,0.390244,0.578947,1.0,1.000000,-0.542857,-0.053763,1.0,-1.00000,-1.0
1,21057,0.451613,-0.185185,1.0,0.292683,0.789474,1.0,1.000000,-0.371429,-0.204301,1.0,-0.99811,-1.0
2,9654,0.290323,1.000000,1.0,0.243902,0.578947,1.0,1.000000,-0.200000,-0.118280,1.0,-1.00000,-1.0
3,19535,0.193548,0.111111,1.0,0.317073,0.789474,1.0,1.000000,-0.485714,-0.161290,1.0,-1.00000,-1.0
4,11803,0.290323,1.000000,1.0,0.390244,0.578947,1.0,0.959596,-0.600000,-0.010753,1.0,-1.00000,-1.0


In [4]:
y_train=pd.read_csv('/Users/zhilinyang/Desktop/work/y_train.csv',header=None)
y_train=y_train.iloc[:,1:]
y_train.head()

,1
0,0
1,0
2,0
3,0
4,0


In [5]:
y_test=pd.read_csv('/Users/zhilinyang/Desktop/work/y_test.csv',header=None)
y_test=y_test.iloc[:,1:]
y_test.head()

,1
0,0
1,0
2,0
3,0
4,0


In [136]:
from sklearn.neural_network import MLPClassifier

In [177]:
clf = MLPClassifier(activation='tanh',alpha=0.05,hidden_layer_sizes=(20, 1), max_iter=400, learning_rate_init=0.1)

In [178]:
clf.fit(train_df.iloc[:,1:], y_train)

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 1), learning_rate='constant',
       learning_rate_init=0.1, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [179]:
y_pred=clf.predict(test_df.iloc[:,1:])

In [19]:
from sklearn.metrics import confusion_matrix
def FDR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    TP=mat[1,1]
    FN=mat[1,0]
    FDR=TP/(TP+FN)
    return FDR

In [20]:
def FAR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    FP=mat[0,1]
    TN=mat[0,0]
    FAR=FP/(FP+TN)
    return FAR

In [21]:
def predict_disk(testset,y_pred,disk_name_list):
    result=[]
    test_set=testset.groupby('SID')
    y_pred=pd.Series(y_pred)
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_pred.reindex(sample_index)
        sample_result=pd.Series(sample_result)
        if sample_result.any():
            result.append(1)
        else:
            result.append(0)
    return result

In [25]:
def disk_state(testset,y_true,disk_name_list):
    result=[]
    test_set=testset.groupby('SID')
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_true.reindex(sample_index)
        sample_result=sample_result.values
        if sample_result.sum():
            result.append(1)
        else:
            result.append(0)
    return result

In [22]:
def disk_FDR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['SID'].unique())
    true_state=disk_state(x_test,y_test,x_test['SID'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FDR(true_state, disk_pred)
    return diskfdr

In [23]:
def disk_FAR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['SID'].unique())
    true_state=disk_state(x_test,y_test,x_test['SID'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FAR(true_state, disk_pred)
    return diskfdr

In [26]:
disk_FDR(test_df,y_pred,y_test) #For DT

0.91538461538461535

In [27]:
disk_FAR(test_df,y_pred,y_test) #For DT

0.0046163226199808383

In [149]:
disk_FDR(test_df,y_pred,y_test) #For NN

0.7846153846153846

In [150]:
disk_FAR(test_df,y_pred,y_test) #For NN

0.016331330023517117

In [2]:
good_disks=pd.read_csv('/Users/zhilinyang/Desktop/work/good_disks.csv')
good_disks=good_disks.iloc[:,1:]
good_disks.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,434,1,0.290323,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,-0.182796,1.0,-0.964083,-1.0
1,434,1,0.322581,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.485714,-0.161290,1.0,-0.964083,-1.0
2,434,1,0.354839,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.485714,-0.139785,1.0,-0.964083,-1.0
3,434,1,0.354839,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.485714,-0.182796,1.0,-0.964083,-1.0
4,434,1,0.387097,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,-0.182796,1.0,-0.964083,-1.0


In [25]:
def change_rate(dataset,feature_list,rate,new_feature_list):
    num=1
    disk_list=dataset['SID'].unique()
    groups=dataset.groupby('SID')
    new_dataset=pd.DataFrame()
    col=list(dataset.columns)+new_feature_list
    for disk_id in disk_list:
        disk=groups.get_group(disk_id)
        part=disk[feature_list]
        new_sample=pd.DataFrame()
        for hour in range(rate,len(part)):
            temp=abs((part.iloc[hour]-part.iloc[hour-rate])).values
            new=np.concatenate((disk.iloc[hour], temp), axis=0)
            new=pd.Series(new,index=col)
            new_sample=new_sample.append(new,ignore_index=True)
        new_dataset=new_dataset.append(new_sample)
        newcolname=["SID","label","SMART_1","SMART_3","SMART_5","SMART_7","SMART_9","SMART_187",
         "SMART_189","SMART_194","SMART_195","SMART_197","RAW_5","RAW_197"]+new_feature_list
        new_dataset=new_dataset[newcolname]
        num=num+1
        print(num)
    return new_dataset

In [26]:
len(good_disks['SID'].unique())

22962

In [ ]:
new_set=change_rate(good_disks,["SMART_1","SMART_5","SMART_187","SMART_195","SMART_197","RAW_5","RAW_197"],6,["cg_1","cg_5","cg_187","cg_195","cg_197","cgr_5","cgr_197"])
new_set.head(20)

In [28]:
fail_disks=pd.read_csv('/Users/zhilinyang/Desktop/work/fail_disks.csv')
fail_disks=fail_disks.iloc[:,1:]
fail_disks.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197
0,1,-1,0.483871,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0
1,1,-1,0.000000,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0
2,1,-1,0.096774,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.714286,-0.139785,1.0,0.935255,-1.0
3,1,-1,0.193548,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.139785,1.0,0.935255,-1.0
4,1,-1,0.290323,0.259259,-1.0,0.317073,0.894737,0.878788,0.89899,-0.657143,-0.118280,1.0,0.935255,-1.0


In [ ]:
new_set_fail=change_rate(fail_disks,["SMART_1","SMART_5","SMART_187","SMART_195","SMART_197","RAW_5","RAW_197"],6,["cg_1","cg_5","cg_187","cg_195","cg_197","cgr_5","cgr_197"])
new_set_fail.head(20)

In [30]:
len(fail_disks["SID"].unique())

433

In [34]:
healthy_train2=pd.DataFrame()
healthy_train2=get_healthy_train(new_set,healthy_train2)
healthy_train2.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,...,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
3,434.0,1.0,0.451613,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.964083,-1.0,0.096774,0.0,0.0,0.000000,0.0,0.0,0.0
4,434.0,1.0,0.483871,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.964083,-1.0,0.096774,0.0,0.0,0.021506,0.0,0.0,0.0
11,434.0,1.0,0.387097,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.964083,-1.0,0.258065,0.0,0.0,0.021506,0.0,0.0,0.0
67,435.0,1.0,0.483871,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.428571,...,1.0,-0.996692,-1.0,0.064516,0.0,0.0,0.021506,0.0,0.0,0.0
79,435.0,1.0,0.419355,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.485714,...,1.0,-0.996692,-1.0,0.096774,0.0,0.0,0.000000,0.0,0.0,0.0


In [45]:
len(healthy_train2)

68886

In [37]:
healthy_test2=pd.DataFrame()
healthy_test2=get_healthy_test(new_set,healthy_test2)
healthy_test2.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,...,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
123,434.0,1.0,0.387097,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.600000,...,1.0,-0.964083,-1.0,0.096774,0.0,0.0,0.021505,0.0,0.0,0.0
117,434.0,1.0,0.290323,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.964083,-1.0,0.354839,0.0,0.0,0.064516,0.0,0.0,0.0
134,434.0,1.0,0.290323,1.0,0.979798,0.536585,0.157895,1.0,-1.0,-0.600000,...,1.0,-0.964083,-1.0,0.161290,0.0,0.0,0.043011,0.0,0.0,0.0
138,435.0,1.0,0.354839,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.996692,-1.0,0.129033,0.0,0.0,0.064516,0.0,0.0,0.0
147,435.0,1.0,0.129032,1.0,1.000000,0.512195,0.157895,1.0,-1.0,-0.542857,...,1.0,-0.996692,-1.0,0.258065,0.0,0.0,0.107527,0.0,0.0,0.0


In [46]:
len(healthy_test2)

68817

In [41]:
fail_disks_num=len(new_set_fail["SID"].unique())
fail_samples2=pd.DataFrame()
fail_samples2=get_lastn_samples(new_set_fail,fail_samples2,12,fail_disks_num)
fail_samples2.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,...,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
122,1.0,-1.0,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.000000,0.0,0.0,0.0
123,1.0,-1.0,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.000000,0.0,0.0,0.0
124,1.0,-1.0,0.483871,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.021506,0.0,0.0,0.0
125,1.0,-1.0,-0.064516,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.483871,0.0,0.0,0.086022,0.0,0.0,0.0
126,1.0,-1.0,0.032258,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.600000,...,1.0,0.935255,-1.0,0.387097,0.0,0.0,0.064516,0.0,0.0,0.0


In [42]:
fail_list2=pd.DataFrame(fail_samples2['SID'].unique())
train_list2=fail_list2.sample(frac=0.7).values

In [44]:
fail_train2=pd.DataFrame()
groups=fail_samples2.groupby('SID')
for sid in fail_samples2['SID'].unique():
    if sid in train_list2:
        obj=groups.get_group(sid)
        fail_train2=fail_train2.append(obj)
fail_train2.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,...,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
122,1.0,-1.0,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.000000,0.0,0.0,0.0
123,1.0,-1.0,0.451613,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.000000,0.0,0.0,0.0
124,1.0,-1.0,0.483871,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.064516,0.0,0.0,0.021506,0.0,0.0,0.0
125,1.0,-1.0,-0.064516,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.542857,...,1.0,0.935255,-1.0,0.483871,0.0,0.0,0.086022,0.0,0.0,0.0
126,1.0,-1.0,0.032258,0.259259,-1.0,0.341463,0.789474,0.878788,0.878788,-0.600000,...,1.0,0.935255,-1.0,0.387097,0.0,0.0,0.064516,0.0,0.0,0.0


In [51]:
fail_test2=pd.DataFrame()
groups=fail_samples2.groupby('SID')
train_list2=list(fail_train2['SID'].unique())
for sid in fail_samples2['SID'].unique():
    if sid not in train_list2:
        obj=groups.get_group(sid)
        fail_test2=fail_test2.append(obj)
fail_test2.head()

,SID,label,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,...,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
461,15.0,-1.0,0.290323,1.0,-1.0,0.560976,0.052632,1.0,1.0,-0.714286,...,1.0,0.935255,-1.0,0.129032,0.0,0.0,0.021506,0.0,0.0,0.0
462,15.0,-1.0,0.322581,1.0,-1.0,0.560976,0.052632,1.0,1.0,-0.771429,...,1.0,0.935255,-1.0,0.129032,0.0,0.0,0.043011,0.0,0.0,0.0
463,15.0,-1.0,0.354839,1.0,-1.0,0.560976,0.052632,1.0,1.0,-0.771429,...,1.0,0.935255,-1.0,0.096774,0.0,0.0,0.000000,0.0,0.0,0.0
464,15.0,-1.0,0.387097,1.0,-1.0,0.560976,0.052632,1.0,1.0,-0.771429,...,1.0,0.935255,-1.0,0.290323,0.0,0.0,0.021506,0.0,0.0,0.0
465,15.0,-1.0,0.387097,1.0,-1.0,0.560976,0.052632,1.0,1.0,-0.771429,...,1.0,0.935255,-1.0,0.193549,0.0,0.0,0.000000,0.0,0.0,0.0


In [48]:
train_df2=healthy_train2.append(fail_train2)
train_df2=train_df2.sample(frac=1)
y_train2=train_df2['label']
train_df2=train_df2.drop('label',axis=1)
train_df2.head()

,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
85,11868.0,0.258065,0.555556,1.0,0.390244,0.578947,1.00000,1.000000,-0.600000,-0.075269,1.0,-1.000000,-1.000000,0.129032,0.0,0.0,0.043011,0.0,0.0,0.0
22,6988.0,0.387097,1.000000,1.0,0.512195,0.157895,1.00000,1.000000,-0.257143,-0.032258,1.0,-0.999527,-1.000000,0.258065,0.0,0.0,0.021505,0.0,0.0,0.0
56,21300.0,0.419355,0.777778,1.0,0.317073,0.789474,1.00000,0.636364,-0.428571,-0.010753,1.0,-1.000000,-1.000000,0.129032,0.0,0.0,0.021506,0.0,0.0,0.0
468,307.0,0.193548,0.555556,1.0,0.560976,-0.052632,0.79798,0.939394,-0.828571,-0.053763,1.0,-0.991021,-0.997753,0.129033,0.0,0.0,0.000000,0.0,0.0,0.0
56,15293.0,0.290323,0.555556,1.0,0.414634,0.578947,1.00000,1.000000,-0.542857,-0.053763,1.0,-1.000000,-1.000000,0.129032,0.0,0.0,0.000000,0.0,0.0,0.0


In [ ]:
y_train2[y_train2==1]=0
y_train2[y_train2==-1]=1

In [52]:
test_df2=healthy_test2.append(fail_test2)
test_df2=test_df2.sample(frac=1)
y_test2=test_df2['label']
test_df2=test_df2.drop('label',axis=1)
test_df2.head()

,SID,SMART_1,SMART_3,SMART_5,SMART_7,SMART_9,SMART_187,SMART_189,SMART_194,SMART_195,SMART_197,RAW_5,RAW_197,cg_1,cg_5,cg_187,cg_195,cg_197,cgr_5,cgr_197
125,4475.0,0.290323,1.000000,1.0,0.536585,0.052632,1.0,1.000000,-0.714286,-0.161290,1.0,-0.997637,-1.0,0.161290,0.0,0.0,0.086021,0.0,0.0,0.0
123,14671.0,0.387097,0.555556,1.0,0.390244,0.578947,1.0,0.030303,-0.600000,-0.010753,1.0,-1.000000,-1.0,0.032258,0.0,0.0,0.064516,0.0,0.0,0.0
137,19920.0,0.419355,0.259259,1.0,0.390244,0.789474,1.0,1.000000,-0.485714,-0.032258,1.0,-1.000000,-1.0,0.064516,0.0,0.0,0.021505,0.0,0.0,0.0
123,2785.0,0.387097,1.000000,1.0,0.560976,0.052632,1.0,1.000000,-0.600000,-0.290323,1.0,-0.997637,-1.0,0.161291,0.0,0.0,0.021505,0.0,0.0,0.0
148,13665.0,0.387097,0.555556,1.0,0.414634,0.578947,1.0,1.000000,-0.371429,-0.096774,1.0,-1.000000,-1.0,0.193549,0.0,0.0,0.021505,0.0,0.0,0.0


In [53]:
y_test2[y_test2==1]=0
y_test2[y_test2==-1]=1

In [54]:
train_df2.to_csv('/Users/zhilinyang/Desktop/work/train_df2.csv')
test_df2.to_csv('/Users/zhilinyang/Desktop/work/test_df2.csv')
y_train2.to_csv('/Users/zhilinyang/Desktop/work/y_train2.csv')
y_test2.to_csv('/Users/zhilinyang/Desktop/work/y_test2.csv')